In [31]:
import cassiopeia as cass
from riot_api import get_puuid
cass.set_riot_api_key('RGAPI-28887941-ef83-4b09-869e-a51fd2e2b671')
api_key = 'RGAPI-28887941-ef83-4b09-869e-a51fd2e2b671'

puuid = get_puuid('kitkat','miffy','americas', api_key=api_key)
a_summoner = cass.get_summoner(puuid=puuid,region="NA")

https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/kitkat/miffy?api_key=RGAPI-28887941-ef83-4b09-869e-a51fd2e2b671


In [51]:
match = a_summoner.match_history[21]

for participant in match.participants:
    if participant.summoner == a_summoner:
        print(participant.lane)
        print(participant.champion.name)
        for i, event in enumerate(participant.timeline.events):
            print(f"Type: {event.type}, time: {event.timestamp}")
            if event.type in {"CHAMPION_KILL", "ELITE_MONSTER_KILL", "BUILDING_KILL"}:
                print(event.position)


Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/cmDcscqjKchTj0kJnY176BcBRtLWANM_ZGf_GHmZuD5cEe3lb-MukzvrfDc7GJmTb4IRGSW6Xh6kUA/ids?start=0&count=100
Lane.utility
Lux
Type: ITEM_PURCHASED, time: 0:00:12.762000
Type: ITEM_PURCHASED, time: 0:00:12.762000
Type: ITEM_PURCHASED, time: 0:00:12.762000
Type: SKILL_LEVEL_UP, time: 0:01:49.568000
Type: WARD_PLACED, time: 0:02:00.863000
Type: ITEM_DESTROYED, time: 0:02:11.890000
Type: LEVEL_UP, time: 0:02:25.521000
Type: SKILL_LEVEL_UP, time: 0:02:26.590000
Type: CHAMPION_KILL, time: 0:02:31.801000
<Position (12833, 1905)>
Type: CHAMPION_SPECIAL_KILL, time: 0:02:31.801000
Type: ITEM_DESTROYED, time: 0:02:32.702000
Type: CHAMPION_KILL, time: 0:02:36.446000
<Position (10823, 1770)>
Type: LEVEL_UP, time: 0:02:51.109000
Type: SKILL_LEVEL_UP, time: 0:02:52.111000
Type: ITEM_PURCHASED, time: 0:03:06.980000
Type: ITEM_PURCHASED, time: 0:03:07.448000
Type: WARD_PLACED, time: 0:04:33.466000
Type: LEVEL_UP, time: 0:04:38.910000

In [36]:
match = a_summoner.match_history[5]

timeline = match.timeline

for participant in match.participants:
    if participant.summoner == a_summoner:
        id = participant.id

for frame in timeline.frames:
    participant_frame = frame.participant_frames[id]
    print(f"CS: {participant_frame.creep_score}, Gold: {participant_frame.gold_earned}, XP: {participant_frame.experience}, Score: {participant_frame.position}")

        
        

Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/cmDcscqjKchTj0kJnY176BcBRtLWANM_ZGf_GHmZuD5cEe3lb-MukzvrfDc7GJmTb4IRGSW6Xh6kUA/ids?start=0&count=100
CS: 0, Gold: 500, XP: 0, Score: <Position (554, 581)>
CS: 0, Gold: 500, XP: 0, Score: <Position (1485, 11773)>
CS: 3, Gold: 595, XP: 185, Score: <Position (2180, 12120)>
CS: 12, Gold: 917, XP: 924, Score: <Position (2939, 13095)>
CS: 17, Gold: 1156, XP: 1294, Score: <Position (1350, 10994)>
CS: 24, Gold: 1397, XP: 1849, Score: <Position (2025, 11980)>
CS: 28, Gold: 1610, XP: 2249, Score: <Position (2794, 13064)>
CS: 30, Gold: 1812, XP: 2405, Score: <Position (1899, 11324)>
CS: 35, Gold: 2026, XP: 2962, Score: <Position (954, 10890)>
CS: 40, Gold: 2246, XP: 3394, Score: <Position (1827, 11853)>
CS: 45, Gold: 2577, XP: 3670, Score: <Position (1195, 11420)>
CS: 49, Gold: 2763, XP: 3987, Score: <Position (2850, 12462)>
CS: 56, Gold: 3024, XP: 4325, Score: <Position (898, 10736)>
CS: 58, Gold: 3187, XP: 4506, Score

In [31]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

pd.set_option('display.max_rows', 100) 

df_normal = pd.read_csv("./match_info_csvs/kitkat_miffy_Kai'Sa_match_info.csv")

df_normal = df_normal.drop(df_normal.columns[0], axis=1).drop(columns=['match_id', 'nexus_lost'])

features_normal = df_normal.drop('win', axis=1)
target_normal = df_normal['win']

X_train_normal, X_test_normal, y_train_normal, y_test_normal = train_test_split(features_normal, target_normal, test_size=0.2, random_state=42)

rf_normal = RandomForestClassifier(n_estimators=500, random_state=301)
rf_normal.fit(X_train_normal, y_train_normal)
y_pred_normal = rf_normal.predict(X_test_normal)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=30), param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_normal, y_train_normal)

rf_best = grid_search.best_estimator_

y_pred_normal = rf_best.predict(X_test_normal)
print(classification_report(y_test_normal, y_pred_normal))

importances_normal = rf_best.feature_importances_
feature_names_normal = features_normal.columns

importance_df_normal = pd.DataFrame({
    'Feature': feature_names_normal,
    'Importance': importances_normal
})

importance_df_normal = importance_df_normal.sort_values(by='Importance', ascending=False)
print(importance_df_normal)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END ma

In [34]:
df_pro = pd.read_csv("./pro_match_info_csvs/아따머스마야_kr1_Kai'Sa_match_info.csv")
df_pro = df_pro.drop(df_pro.columns[0], axis=1).drop(columns=['match_id', 'nexus_lost'])

features_pro = df_pro.drop('win', axis=1)
target_pro = df_pro['win']

X_train_pro, X_test_pro, y_train_pro, y_test_pro = train_test_split(features_pro, target_pro, test_size=0.2, random_state=2)

rf_pro = RandomForestClassifier(n_estimators=500, random_state=2)
rf_pro.fit(X_train_pro, y_train_pro)
y_pred_pro = rf_pro.predict(X_test_pro)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=30), param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_pro, y_train_pro)

rf_best = grid_search.best_estimator_

y_pred_pro = rf_best.predict(X_test_pro)
print(classification_report(y_test_pro, y_pred_pro))

importances_pro= rf_best.feature_importances_
feature_names_pro = features_pro.columns

importance_df_pro = pd.DataFrame({
    'Feature': feature_names_pro,
    'Importance': importances_pro
})

importance_df_pro = importance_df_pro.sort_values(by='Importance', ascending=False)
print(importance_df_pro)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END ma

/Users/jasonzhao/Desktop/Mundo-Car/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estim